# Прием и обработка твитов микробатчем

## Инициализация

In [1]:
import org.apache.spark.sql.types.{StructType, StringType, IntegerType, TimestampType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.toree.kernel.api
import java.util.Calendar

In [21]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


## Чтение модели

In [7]:
val modelPath = "/home/jovyan/models/spark-ml-model"
val model = PipelineModel.load(modelPath)

modelPath = /home/jovyan/models/spark-ml-model
model = pipeline_e3869da6fdc9


pipeline_e3869da6fdc9

## Определяем схему и инициируем потоковый датафрейм

In [16]:
val inputStreamPath = "/home/jovyan/work/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("hiddentargetclue", IntegerType)
    .add("arrived_key", StringType)
    .add("timestamp", TimestampType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/work/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true), StructField(hiddentargetclue,IntegerType,true), StructField(arrived_key,StringType,true), StructField(timestamp,TimestampType,true))
inputDF = [tweet: string, hiddentargetclue: int ... 2 more fields]


[tweet: string, hiddentargetclue: int ... 2 more fields]

## Глобальная переменная для удобства просмотра датасета в отдельной ячейке

In [17]:
var globDF:DataFrame = null

globDF: org.apache.spark.sql.DataFrame = null


## Внимание!
- К сожалению, я не нашел внятной доки по апи ядра toree, чтобы нормально выводить оперативно изменяющийся датасет в этом ноутбуке
- В связи с этим см. след. пункт
- В блоке приема твитов ниже вывод датасета для удобства просмотра осуществляется через глобальную переменную globDF, которая просматривается при помощи выполнения блока, следующего за приемом твитов

## Микробатч приема твитов

In [30]:
// Определяем udf для получения probability по 0 и 1
val getProbability =
    udf(
        (prediction: org.apache.spark.ml.linalg.Vector, pos: Integer) =>
        {
            prediction(pos)
        }
    )

var fRuns = 0

// Микробатч для вывода результата предсказания
// Выводится вероятность негативного твита
// В задании написано, что это последняя колонка, но она здесь вроде первая (в позиции 0)
val stream = inputDF.writeStream.foreachBatch {
    (batchDF: DataFrame, batchId: Long) => {
        try {
            fRuns += 1
            print(s"${Calendar.getInstance().toInstant} - loaded from the events stream $fRuns times"+13.toChar)
            // Применяем модель и получаем соотв. датасет с предсказаниями
            globDF = 
                model.transform(batchDF)
                    .select(
                        $"arrived_key",
                        $"timestamp",
                        $"tweet",
                        // $"hiddentargetclue",
                        (getProbability($"probability",lit(0))).alias("Negative Probability")
                    )
        } catch {
            case e:Throwable => {
                print(e.getMessage.replaceAll("\n"," "))
                print(13.toChar)
            }
        }
    }
}.start()

getProbability = UserDefinedFunction(<function2>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7, IntegerType)))
fRuns = 0
stream = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@69d1b32e


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@69d1b32e

## Блок просмотра результата
- Этот блок предназначен для просмотра результата потоковой обработки твитов и применения модели из предыдущего блока
- Каждый раз при выполнении этого блока будет выводиться оперативное состояние датасета из блока выше

In [31]:
%%dataframe --limit=100
globDF

arrived_key,timestamp,tweet,Negative Probability
2020-01-27 21:30:20.831 - 09177,2020-01-27 23:30:20.831,Gardening faery does not exist.,0.49339774930100216
2020-01-27 21:30:20.831 - 22574,2020-01-27 23:30:20.831,@Fyt1247 i heard ure officially gay now Battty gyaaal,0.49862572525421706
2020-01-27 21:30:20.831 - 26613,2020-01-27 23:30:20.831,"@mileycyrus YOU WONNN???? Ayayay congrats!! You deserve it Im from Indonesia and we have different time. So, I didnt watch you. Sorry",0.5022639106298054
2020-01-27 21:30:20.831 - 35416,2020-01-27 23:30:20.831,DAMN ALLERGIES! its so itchy. :|:| i think its just an insect bite tho. but its so itchy!!!! i hate it.,0.542921695640404
2020-01-27 21:30:20.831 - 46796,2020-01-27 23:30:20.831,@xSilja I am pretty sure you are better than me. I would never be anywhere near getting 10. I suck so bad and my teacher is wicked,0.5059547803599953
2020-01-27 21:30:20.831 - 58875,2020-01-27 23:30:20.831,I feel like I don't know any of this stuff.,0.5098724112223645
2020-01-27 21:30:20.831 - 77773,2020-01-27 23:30:20.831,my baby is getting old he had a breathing attack and scared me. makes me want to cry.,0.5485581169759849
2020-01-27 21:30:20.831 - 78685,2020-01-27 23:30:20.831,I can't take it anymore I really need a memory stick,0.5233636405528105
2020-01-27 21:30:20.831 - 11235,2020-01-27 23:30:20.831,@smallerrock It is very good! I was impressed,0.4994158314449909
2020-01-27 21:30:20.831 - 12002,2020-01-27 23:30:20.831,@TheRealLuis one i wrote and post in YouTube,0.5152059536821942


## Останов чтения потока

In [32]:
globDF = null
stream.stop()

globDF: org.apache.spark.sql.DataFrame = null
